In [2]:
from sklearn.cross_validation import KFold,StratifiedKFold
from gensim.models import word2vec
import xgboost as xgb
import numpy as np 
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import csv

In [5]:
add = u'G:/NLP/搜狗复赛数据/user_tag_query.10W.TRAIN'
csvfile = file(add+'.csv','wb') #
writer = csv.writer(csvfile)
writer.writerow(['ID', 'age', 'Gender', 'Education', 'QueryList'])
with open(add,'r') as f:
    for line in f:
        line.strip()
        data = line.split("\t")
        writedata = [data[0],data[1],data[2],data[3]]
        querystr = ''
        data[-1] = data[-1][:-1]
        for d in data[4:]:
           try:
                querystr += d.decode('GB18030').encode('utf8') + '\t'
           except:
               print (data[0],querystr)
        querystr = querystr[:-1]
        writedata.append(querystr)
        writer.writerow(writedata)
       

In [6]:
add = u'G:/NLP/搜狗复赛数据/user_tag_query.10W.TEST'#path of the original test file

csvfile = file(add + '.csv', 'wb')# the path of the generated test file
writer = csv.writer(csvfile)
writer.writerow(['ID', 'QueryList'])
with open(add, 'r') as f:
    for line in f:
        data = line.split("\t")
        writedata = [data[0]]
        querystr = ''
        data[-1]=data[-1][:-1]
        for d in data[1:]:
           try:
                querystr += d.decode('GB18030').encode('utf8') + '\t'
           except:
               print (data[0],querystr)
        querystr = querystr[:-1]
        writedata.append(querystr)
        writer.writerow(writedata) 

In [3]:
class w2v():
    def __init__(self,size=300):
        random_rate = 8240
        self.size = size
        self.svc = SVC(C=1,random_state=random_rate)
        self.LR = LogisticRegression(C=1.0,max_iter=100,class_weight='balanced', random_state=random_rate, n_jobs=-1)
        self.clf = LinearSVC(random_state=random_rate)
        
    def fit(self,X,Y,T):
        """
        train and predict
        """
        print("fitting...")
        self.LR.fit(X,Y)
        res = self.LR.predict(T)
        return res
    
    def validation(self,X,Y,kind):
        """
        使用2-fold进行验证
        """
        fold_n = 2
        folds = list(StratifiedKFold(Y,n_folds=fold_n,random_state=0))
        score = np.zeros(fold_n)
        for j,(train_idx,test_idx) in enumerate(folds):
            print(j+1,'-fold')
            X_train = X[train_idx]
            y_train = Y[train_idx]
            X_test = X[test_idx]
            y_test = Y[test_idx]
            
            res = self.fit(X_train,y_train,X_test)
            cur = sum(y_test==res)*1.0/len(res)
            score[j] = cur
            
        print(score,score.mean())
        return score.mean()
    
    def train_w2v(self,filename):
        """
        训练wv模型
        """
        sentences = word2vec.LineSentence(filename) # 加载语料，要求语料为“一行一文本”的格式
        print('正在训练w2v针对语料：',str(filename))
        print('size is:',self.size)
        model = word2vec.Word2Vec(sentences,size=self.size,window=100,workers=48) #训练模型，注意参数window对结果有影响，一般是5-100
        savepath = '20w_size_win100_'+str(self.size)+'.model' #保存model
        print("训练完毕，已经保存：",savepath)
        model.save(savepath)
        
    def load_transform(self,X):
        """
        载入模型，并且声称wv向量
        """
        print('载入模型中')
        model = word2vec.Word2Vec.load('20w_size_win100_300.model') #填入自己的路径
#         print{"载入成功"}
        res = np.zeros((len(X),self.size))
        for i,line in enumerate(X):
            line = line.decode('utf-8')
            terms = line.split()
            count = 0
            for j,term in enumerate(terms):
                try: #try失败说明X中有单词不在model，训练的时候model的模型是min_count的，忽略了一部分单词
                    count+=1
                    res[i]+=np.asarray(model[term])
                except:
                    1 == 1
            if count!=0:
                res[i] = res[i]/float(count) #求均值
        return res
 

In [4]:
import numpy as np
class preprocess():
    # 主要功能：去除缺失值
    def removezero(self, x, y):
        nozero = np.nonzero(y)
        y = y[nozero]
        x = np.array(x)
        x = x[nozero]
        return x, y


In [5]:
import csv

In [6]:
def input(trainname):
    """
    load file
    """
    traindata = []
    with open(trainname,'rb') as f:
        reader = csv.reader(f)
        count = 0
        for line in reader:
            try:
                traindata.append(line[0])
                count+=1
            except:
                print("error:",line,count)
                traindata.append(" ")
    return traindata


In [ ]:
if __name__==='__main__':
    """
    使用方法：先讯洗洗脸wv的model，然后在生成wv的向量，最后可以使用2-fold验证
    主要目的：生成wv向量，提供给下一个步骤，特征融合
    注意路径
    """
    print("------w2v-----")
    order = 'test'
    classob = w2v(300)
    
    if order=="train w2v model": #训练wv的model
        totalname = 'jieba_total_cut.csv' #纯文本文件路径
        classod.train_w2v(totalname)
        exit()
    elif order=='getvec': #利用生成的model得到文档的WV的向量，使用求和平均法
        trainname = 'jieba_train_cut.csv'
        testname = 'jieba_test_cut.csv'
        traindata = input(trainname)
        testdata = input(testname)
        res1 = classob.load_trainsform(traindata)
        res2 = classob.load_trainsform(testdata)
        np.save()
        print(res1.shape,res2.shape)
        np.save('wv300_win100.train.npy', res1)#保存生成的向量
        np.save('wv300_win100.test.npy', res2)
        exit()
        
    #以下为测试WV向量，即仅仅使用WV向量做这个比赛，目的在与寻找好的参数的WV向量
    print '载入所有的w2v向量中..'
    w2vtrain = np.load('wv300_win100.train.npy')
    w2vtest = np.load('wv300_win100.test.npy')

    #防止出现非法值
    if np.any((np.isnan(w2vtrain))):
        print 'nan to num!'
        w2vtrain = np.nan_to_num(w2vtrain)

    if np.any((np.isnan(w2vtest))):
        print 'nan to num!'
        w2vtest = np.nan_to_num(w2vtest)

    #载入label文件
    label_genderfile_path = 'train_gender.csv'
    label_agefile_path = 'train_age.csv'
    label_edufile_path = 'train_education.csv'
    genderdata = np.loadtxt(open(label_genderfile_path, 'r')).astype(int)
    agedata = np.loadtxt(open(label_agefile_path, 'r')).astype(int)
    educationdata = np.loadtxt(open(label_edufile_path, 'r')).astype(int)

    print '预处理中..'
    preprocessob = preprocess.preprocess()
    gender_traindatas, genderlabel = preprocessob.removezero(w2vtrain, genderdata)
    age_traindatas, agelabel = preprocessob.removezero(w2vtrain, agedata)
    edu_traindatas, edulabel = preprocessob.removezero(w2vtrain, educationdata)
    # ------------------------------------------------------

    if order == 'test': #使用2-fold进行验证
        res1 = classob.validation(gender_traindatas, genderlabel, kind='gender')
        res2 = classob.validation(age_traindatas, agelabel, kind='age')
        res3 = classob.validation(edu_traindatas, edulabel, kind='edu')
        print 'avg is:', (res1+res2+res3)/3.0
    else:
        print 'error!'
        exit()
       

In [8]:
import multiprocessing,Queue
from sklearn.cross_validation import KFold, StratifiedKFold
import xgboost as xgb
from STFIWF import TfidfVectorizer
import numpy as np
from sklearn.linear_model import SGDClassifier, LogisticRegression,RidgeClassifier,PassiveAggressiveClassifier,Lasso,HuberRegressor
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import VotingClassifier,RandomForestClassifier,gradient_boosting
from sklearn.ensemble.bagging import BaggingClassifier
from sklearn.ensemble.weight_boosting import AdaBoostClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler

ImportError: No module named 'Queue'